In [100]:
import pandas as pd
df=pd.read_csv("/Users/srinjoybhuiya/PycharmProjects/funfMRI/imagenet_kamitani_subset/image_training_id.csv", header=None).astype(str)
df.columns=["img_id", "file_name"]
df["category_id"]=df["file_name"].apply(lambda x: x.split("_")[0]).apply(lambda x: x[1:])
df["img_name"]=df["file_name"].apply(lambda x: x.split("_")[1][:-5])
df["feature_dict_keys"]=df["category_id"]+"_"+df["img_name"]


train_category_ids=df["category_id"].unique().astype(list)

df

,img_id,file_name,category_id,img_name,feature_dict_keys
0,1518878.010042,n01518878_10042.JPEG,01518878,10042,01518878_10042
1,1518878.012028,n01518878_12028.JPEG,01518878,12028,01518878_12028
2,1518878.014075,n01518878_14075.JPEG,01518878,14075,01518878_14075
3,1518878.01491,n01518878_14910.JPEG,01518878,14910,01518878_14910
4,1518878.005958,n01518878_5958.JPEG,01518878,5958,01518878_5958
...,...,...,...,...,...
1195,13111881.028475,n13111881_28475.JPEG,13111881,28475,13111881_28475
1196,13111881.006054,n13111881_6054.JPEG,13111881,6054,13111881_6054
1197,13111881.00656,n13111881_6560.JPEG,13111881,6560,13111881_6560
1198,13111881.007817,n13111881_7817.JPEG,13111881,7817,13111881_7817


In [155]:
from tensordict import TensorDict
import os
import torch
features_root="/Users/srinjoybhuiya/PycharmProjects/funfMRI/cnn_feature_convnext_and_alexnet/"
feature_files=os.listdir(features_root)
print(feature_files)
for file in feature_files:
    avg_img_features_file_name=features_root+file[:-3]+"_avg_img_features.pt"
    file_count=0
    file_meta_data=file[:-3].split("_")
    print(file)
    model_name=file_meta_data[0]
    train_or_test=file_meta_data[2]
    print("##################",model_name, train_or_test,"######################")
    full_file_name=features_root+file
    features=torch.load(full_file_name)
    category_wise_avg_features=dict()
    for category in train_category_ids:
        avg_img_features=dict()
        #print("##################",category,"######################")
        for img_name in features.keys():
            num_img=0
            img_features=features[img_name]
            
            #print(category,img_name.split("_")[0])
            if category==img_name.split("_")[0]:
                file_count+=1
                #print("adding features of img",img_name,"to category",category,"features")
                
                for key in img_features.keys():
                    #print("\t","adding layer",key,"for img",img_name,"to category",category,"features")
                    if key not in avg_img_features.keys():
                        #print("\t \t previously features for this layer were not seen for the category")
                        avg_img_features[key]=img_features[key]
                        num_img+=1
                    else:
                        #print("\t \t previously features for this layer were seen for the category soo adding up with prev features")
                        num_img+=1
                        avg_img_features[key]=avg_img_features[key]+(1/num_img)*(img_features[key]-avg_img_features[key])
                    #print("num_img",num_img)
                
                    
            
        category_wise_avg_features[category]=avg_img_features
    
    avg_category_tensors =TensorDict(category_wise_avg_features,batch_size=1)
    print(avg_img_features_file_name)
    torch.save(avg_category_tensors,avg_img_features_file_name)     
                    
print("###################end of processing###################")                  

         
        

['convnext_features_train.pt', 'alexnet_dict_train.pt']
convnext_features_train.pt
################## convnext train ######################
/Users/srinjoybhuiya/PycharmProjects/funfMRI/cnn_feature_convnext_and_alexnet/convnext_features_train_avg_img_features.pt
alexnet_dict_train.pt
################## alexnet train ######################
/Users/srinjoybhuiya/PycharmProjects/funfMRI/cnn_feature_convnext_and_alexnet/alexnet_dict_train_avg_img_features.pt
###################end of processing###################


1200
dict_keys(['01518878', '01639765', '01645776', '01664990', '01704323', '01726692', '01768244', '01770393', '01772222', '01784675', '01787835', '01833805', '01855672', '01877134', '01944390', '01963571', '01970164', '02005790', '02054036', '02055803', '02068974', '02084071', '02090827', '02131653', '02226429', '02233338', '02236241', '02317335', '02346627', '02374451', '02391049', '02432983', '02439033', '02445715', '02472293', '02480855', '02481823', '02503517', '02508213', '02692877', '02766534', '02769748', '02799175', '02800213', '02802215', '02808440', '02814860', '02841315', '02843158', '02882647', '02885462', '02943871', '02974003', '02998003', '03038480', '03063599', '03079230', '03085013', '03085219', '03187595', '03209910', '03255030', '03261776', '03335030', '03345487', '03359137', '03394916', '03397947', '03400231', '03425413', '03436182', '03445777', '03467796', '03472535', '03483823', '03494278', '03496296', '03512147', '03541923', '03543603', '03544143', '03602883', 

SyntaxError: invalid syntax (1665195085.py, line 2)